<a href="https://colab.research.google.com/github/arlethitgo/GoogleColabNotebooks/blob/main/How_to_Tabulate_PSE_EOD_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PSE EOD Data Extraction Automation using tabula-py

**This script is used to automate extraction of PSE Data from the PSE EOD pdf files into a csv file **

tabula-py is a tool for convert PDF tables to pandas DataFrame. tabula-py is a wrapper of [tabula-java](https://github.com/tabulapdf/tabula-java), which requires java on your machine. tabula-py also enables you to convert tables in a PDF into CSV/TSV files.

tabula-py's PDF extraction accuracy is same as tabula-java or [tabula app](https://tabula.technology/); GUI tool of tabula, so if you want to know the performance of tabula-py, I highly recommend you to try tabula app.

tabula-py is good for:
- automation with Python script
- advanced analytics after converting pandas DataFrame
- casual analytics with Jupyter notebook or Google Colabolatory


## Check Java environment and install tabula-py

tabula-py requires a java environment, so let's check the java environment on your machine.

In [1]:
!java -version

openjdk version "11.0.15" 2022-04-19
OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1)
OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)


After confirming the java environment, install tabula-py by using pip.

In [2]:
# To be more precisely, it's better to use `{sys.executable} -m pip install tabula-py`
!pip install -q tabula-py

     |████████████████████████████████| 12.0 MB 6.0 MB/s 


Before trying tabula-py, check your environment via tabula-py `environment_info()` function, which shows Python version, Java version, and your OS environment.

In [3]:
import tabula

tabula.environment_info()

Python version:
    3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
Java version:
    openjdk version "11.0.15" 2022-04-19
OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1)
OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
tabula-py version: 2.3.0
platform: Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
uname:
    uname_result(system='Linux', node='f0356ad3fcad', release='5.4.188+', version='#1 SMP Sun Apr 24 10:03:06 PDT 2022', machine='x86_64', processor='x86_64')
linux_distribution: ('Ubuntu', '18.04', 'Bionic Beaver')
mac_ver: ('', ('', '', ''), '')
    


## Read a PDF with `read_pdf()` function to read the PSE EOD pdf

tabula-py can load a PDF or file like object on both local or internet by using `read_pdf()` function.

In [6]:
from google.colab import files

import numpy as np
import pandas as pd
import tabula

In [24]:
import requests
from bs4 import BeautifulSoup 
import re
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from time import sleep

### Upload PSE EOD pdf file from https://www.pse.com.ph/market-report/ below

In [7]:
# Upload EOD pdf files
files.upload()

Saving May 06, 2022-EOD.pdf to May 06, 2022-EOD.pdf


{'May 06, 2022-EOD.pdf': b'%PDF-1.4\n%\xe2\xe3\xcf\xd3\n1 0 obj\n<</Producer(\xbd\\n\xaf.\x0b\x0ea\xd1\x8a\xfc=\xd0\x97\xbeDw\xda+\x04\x82\xd3\xe0\xd5\xc4/\x85\xc5\x1d\xbe\xe2\xbe\xb8"H\x1e|Lj\xe2v>\x1e%\xe6{Q\xecT\x1aW\xea\x00f\xfc\xea\xa1\x9f\xe0\xbbS\xa0\x18\x9b\x87:cw\xe6\x811[\x18\xb6\x98\xa80\x15\x9b\x0e2\x8a\xcc\xa8\\(\xa6\x88k\x80Z\x9b\xb3\x9c\x99\xe7fM\xe06\xae\xa8\x9c\xa8\xe06\xb3\x91\xc7\xf1L{k\x80\xd85N\xe7Hh\x89\xb4\xb4=W\x87%\x93\xe9]\xc8\xd2\xf0};\x19W\xe1\x1c\xfa\x1c)/CreationDate(\x90d\xf8fMo9\x85\xc8\xba\x9e\xc5\x93\xa6Am\xc0\\(\x12\x97\xc3y\xc0)/ModDate(\x90d\xf8fMo9\x85\xc8\xba\x9e\xc5\x93\xa6@o\xc0\\(\x12\x97\xc3y\xc0)>>\nendobj\n2 0 obj\n<</Type/Font/Subtype/Type1/BaseFont/Helvetica-Bold/Encoding/WinAnsiEncoding>>\nendobj\n3 0 obj\n<</Length 57/Filter/FlateDecode>>stream\n\x91j\xb7>58\x1f\x9bi8\xe3\xf9U3\xda\xc1\x90d\xe8 k\xd6\xb0\xac\x9a\xfa\xc1\x97\xf4\xbe\xaf\x9e@\xe2\x04:oh\xb1\x13\xa3\xd9\x08\xcccX\xa8\xd7\xc1\x82:\x1a\x84\xd5\x17B\xeb\nendstream\nendobj\n4 0

In [8]:
# Change pdf_path value with the uploaded PSE EOD file.
pdf_path = 'May 06, 2022-EOD.pdf'

dfs = tabula.read_pdf(pdf_path, pages='all', stream=False, multiple_tables=False)
dfs[0].head()


Got stderr: May 06, 2022 2:36:59 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
May 06, 2022 2:36:59 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
May 06, 2022 2:36:59 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
May 06, 2022 2:37:00 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
May 06, 2022 2:37:00 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



,Issue Name Symbol,Bid,Unnamed: 2,Ask,Open,High Low,Unnamed: 6,Close,%Change,Volume,"Value, PHP","Buying/(Selling),"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PHP
1,NaN,NaN,NaN,NaN,NaN,F I N A N C I A L S,NaN,NaN,NaN,NaN,NaN,NaN
2,**** BANKS ****,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ASIA UNITED AUB,43.85,NaN,44.65,44.25,44.8 43.85,NaN,44.65,NaN,"1,500","66,325",-
4,BDO UNIBANK BDO,128,NaN,129.1,128,129.3 127,NaN,128,NaN,"3,082,280","395,689,946","5,334,923"


In [10]:
dfs[0].columns

Index(['Issue Name Symbol', 'Bid', 'Unnamed: 2', 'Ask', 'Open', 'High Low',
       'Unnamed: 6', 'Close', '%Change', 'Volume', 'Value, PHP',
       'Buying/(Selling),'],
      dtype='object')

In [31]:
url = 'https://topforeignstocks.com/stock-lists/the-complete-list-of-listed-companies-in-the-philippines/'
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36"}
session = requests.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

r = session.get(url, headers=headers)
if r.status_code == 200:
    soup = BeautifulSoup(r.text)
else:
    print(f'Error: Status Code {r.status_code}')



In [36]:
pse_code_list = soup.find_all('td', {'class':'column-3'})
pse_symbols = []
for x in pse_code_list:
  pse_symbols.append(x.text)

In [ ]:

# pse_symbols = ['PSEi', 'AC', 'ALI', 'BPI', 'GLO', 'MWC', 'IMI', 'ACEN', 'IPO',
#        'ALLHC', 'AREIT', 'CHI', 'ACEX', 'VIAO*', 'YSH**', 'SM', 'SMPH',
#        'BDO', 'JGS', 'URC', 'RLC', 'RRHI', 'CEB', 'RCR', 'AEV', 'AP', 'UBP', 'Others',
#        'SMC', 'MPI', 'GTCAP', 'AGI', 'MEG', 'MREIT', 'MBT', 'SECB',
#        'TEL', 'CNVRG']

In [167]:
df = dfs[0]
df = df.replace({'PSEI':'PSEi'})
# df['Symbol'] = df['Issue Name Symbol'].apply(lambda x: str(x).split()[-1])
# df['Symbol'].values

In [168]:

df['Symbol'] = df['Issue Name Symbol'].apply(lambda x:list(set(str(x).split()[-1:]) & set(pse_symbols)))
df['Symbol'] = df['Symbol'].apply(lambda x: x[0] if len(x)>0 else np.nan)
df = df.drop_duplicates(subset='Symbol').dropna(subset=['Symbol'])



In [169]:
df = df[df['Symbol'].isin(pse_symbols)]

In [170]:
df


,Issue Name Symbol,Bid,Unnamed: 2,Ask,Open,High Low,Unnamed: 6,Close,%Change,Volume,"Value, PHP","Buying/(Selling),",Symbol
3,ASIA UNITED AUB,43.85,NaN,44.65,44.25,44.8 43.85,NaN,44.65,NaN,"1,500","66,325",-,AUB
4,BDO UNIBANK BDO,128,NaN,129.1,128,129.3 127,NaN,128,NaN,"3,082,280","395,689,946","5,334,923",BDO
6,BANK PH ISLANDS BPI,94.4,NaN,95,97,97 94.4,NaN,94.4,NaN,"1,659,260","158,274,462.5","(44,899,805)",BPI
7,CHINABANK CHIB,28,NaN,28.1,27.75,28 27.65,NaN,28,NaN,"348,500","9,714,190","1,042,415",CHIB
8,CITYSTATE BANK CSB,6.84,NaN,7.99,-,- -,NaN,-,NaN,-,-,-,CSB
...,...,...,...,...,...,...,...,...,...,...,...,...,...
389,KEPWEALTH KPPI,NaN,2.54,2.93,- -,-,-,NaN,-,-,-,NaN,KPPI
390,MAKATI FINANCE MFIN,NaN,2.16,2.69,- -,-,-,NaN,-,-,-,NaN,MFIN
391,MERRYMART MM,NaN,1.61,1.63,1.63 1.63,1.59,1.61,NaN,"1,242,000","1,999,930","48,210",NaN,MM
392,XURPAS X,NaN,0.305,0.315,0.31 0.31,0.305,0.305,NaN,"410,000","127,000",-,NaN,X


In [171]:
df.head()

,Issue Name Symbol,Bid,Unnamed: 2,Ask,Open,High Low,Unnamed: 6,Close,%Change,Volume,"Value, PHP","Buying/(Selling),",Symbol
3,ASIA UNITED AUB,43.85,NaN,44.65,44.25,44.8 43.85,NaN,44.65,NaN,"1,500","66,325",-,AUB
4,BDO UNIBANK BDO,128,NaN,129.1,128,129.3 127,NaN,128,NaN,"3,082,280","395,689,946","5,334,923",BDO
6,BANK PH ISLANDS BPI,94.4,NaN,95,97,97 94.4,NaN,94.4,NaN,"1,659,260","158,274,462.5","(44,899,805)",BPI
7,CHINABANK CHIB,28,NaN,28.1,27.75,28 27.65,NaN,28,NaN,"348,500","9,714,190","1,042,415",CHIB
8,CITYSTATE BANK CSB,6.84,NaN,7.99,-,- -,NaN,-,NaN,-,-,-,CSB


### Extract Data from the Right Columns
Since the file contains multiple tables when extracted, some rows have shifted columns. We need to do some adjustments.

In [191]:
bid_missing = df.loc[(df['Bid'].isna()) & (~df['Close'].isna())].dropna(how='all',axis=1)
if len(bid_missing)>0:
  try:
    bid_missing.columns = ['Issue Name Symbol', 'Bid',  'Ask', 'Open', 'High Low',
        'Close', 'Volume', 'Value, PHP',
        'Buying/(Selling),','Null','Symbol']
  except:
    print('No Stocks with Bid Column Missing')
    bid_missing = pd.DataFrame()
  
bid_missing

,Issue Name Symbol,Bid,Ask,Open,High Low,Close,Volume,"Value, PHP","Buying/(Selling),",Null,Symbol
152,SOLID GROUP SGI,0.88,0.9,0.88,0.9 0.88,0.9,"208,000","184,810","124,460",NaN,SGI
153,SM INVESTMENTS SM,845.5,848,840,848 832,845.5,"906,610","763,599,450","(339,208,760)",NaN,SM
154,SAN MIGUEL CORP SMC,105,106.5,106.5,106.5 104.5,106.5,"75,520","8,012,538","(4,908,195)",NaN,SMC
155,SEAFRONT RES SPM,2.41,2.69,-,- -,-,-,-,-,NaN,SPM
156,TOP FRONTIER TFHI,110.3,115.9,-,- -,-,-,-,-,NaN,TFHI
...,...,...,...,...,...,...,...,...,...,...,...
311,UNITED PARAGON UPM,0.006,0.0063,0.0062,0.0062 0.0061,0.0061,NaN,"12,000,000","73,600",-,UPM
313,ACE ENEXOR ACEX,14.9,15,15.5,15.6 14.8,15,NaN,"1,006,300","15,207,052","(5,469,194)",ACEX
314,ORNTL PETROL A OPM,0.011,0.012,0.011,0.012 0.011,0.012,NaN,"10,000,000","111,000",-,OPM
316,PHILODRILL OV,0.0092,0.0094,0.0091,0.0096 0.0091,0.0094,NaN,"30,000,000","282,000","(18,500)",OV


In [190]:
ask_missing = df.loc[df['Ask'].isna()].dropna(how='all',axis=1)
if len(ask_missing)>0:
  try:
    ask_missing.columns = ['Issue Name Symbol', 'Bid',  'Ask', 'Open', 'High', 'Low',
       'Close', 'Volume', 'Value, PHP',
       'Buying/(Selling),','Symbol']
  except:
    print('No Stocks with Ask Column Missing')
    ask_missing = pd.DataFrame()
ask_missing.head()

,Issue Name Symbol,Bid,Ask,Open,High,Low,Close,Volume,"Value, PHP","Buying/(Selling),",Symbol
50,PILIPINAS SHELL SHLPH,17.22,17.48,17.48,17.48,17.3,17.48,"41,300","718,212","(5,242)",SHLPH
51,SPC POWER SPC,13.94,13.98,13.96,13.98,13.88,13.98,"167,900","2,341,982","582,770",SPC
52,SOLAR PH SPNEC,1.51,1.54,1.49,1.54,1.46,1.54,"58,676,000","88,490,330","(2,020,420)",SPNEC
53,VIVANT VVT,14.24,16.68,-,-,-,-,-,-,-,VVT
55,AGRINURTURE ANI,4.57,4.69,4.73,4.73,4.58,4.69,"1,224,000","5,762,530","5,102,620",ANI


In [174]:
volume_missing = df.loc[df['Volume'].isna()].dropna(how='all',axis=1)
if len(volume_missing)>0:
  try:
    volume_missing.columns = ['Issue Name Symbol', 'Bid',  'Ask', 'Open', 'High Low',
        'Close', 'Volume', 'Value, PHP',
        'Buying/(Selling),','Symbol']
    volume_missing = volume_missing.dropna(subset= ['Buying/(Selling),'])
  except:
    print('No Stocks with Volume Column Missing')
    volume_missing = pd.DataFrame()

volume_missing

""


In [175]:
close_missing = df.loc[df['Close'].isna()].dropna(subset= ['Buying/(Selling),']).dropna(how='all',axis=1)
if len(close_missing)>0:
  try:
    close_missing .columns = ['Issue Name Symbol', 'Bid',  'Ask', 'Open', 'High Low',
        'Close', 'Volume', 'Value, PHP',
        'Buying/(Selling),','Symbol']
  except:
      print('No Stocks with Close Column Missing')
      close_missing = pd.DataFrame()
close_missing.head()

,Issue Name Symbol,Bid,Ask,Open,High Low,Close,Volume,"Value, PHP","Buying/(Selling),",Symbol
102,LMG CORP LMG,3.78,4.39,-,- -,-,-,-,-,LMG
103,MABUHAY VINYL MVC,4.3,4.41,-,- -,-,-,-,-,MVC
104,PRYCE CORP PPC,5.67,5.72,5.62,5.67 5.62,5.67,"3,200","18,099",-,PPC
106,CONCEPCION CIC,19.2,19.98,19.14,19.2 19.12,19.2,"2,100","40,210",-,CIC
107,GREENERGY GREEN,1.49,1.5,1.51,1.54 1.48,1.51,"18,963,000","28,611,030","8,666,810",GREEN


In [188]:
buysell_missing = df.loc[df['Buying/(Selling),'].isna()].loc[df['Bid'].isna()].loc[df['Close'].isna()].dropna(how='all',axis=1)
if len(buysell_missing)>0:
  try:
    buysell_missing .columns = ['Issue Name Symbol', 'Bid',  'Ask', 'Open', 'High Low',
        'Close', 'Volume', 'Value, PHP',
        'Buying/(Selling),','Symbol']
  except:
    print('No Stocks with Buy/Sell Column Missing')
    buysell_missing = pd.DataFrame()
buysell_missing.head()

,Issue Name Symbol,Bid,Ask,Open,High Low,Close,Volume,"Value, PHP","Buying/(Selling),",Symbol
377,ABS HLDG PDR ABSP,12.5,12.66,12.7 12.7,12.7,12.7,"30,000","381,000.00",-,ABSP
378,GMA HLDG PDR GMAP,12.3,12.46,12.48 12.48,12.46,12.46,"1,700","21,188.00","3,744",GMAP
384,ALTUS PROP APVI,16.6,16.7,16.64 16.8,16.64,16.7,"57,300","956,902",-,APVI
386,PHILAB HLDG DNA,-,-,- -,-,-,-,-,-,DNA
388,ITALPINAS IDC,0.87,0.88,0.86 0.88,0.86,0.88,"203,000","175,930",-,IDC


In [186]:
no_missing = df.loc[(~df['Close'].isna()) & (~df['Buying/(Selling),'].isna())].drop(columns = ['Unnamed: 2','Unnamed: 6','%Change'])

if len(no_missing)==0:
  no_missing = pd.DataFrame()
no_missing.head()

,Issue Name Symbol,Bid,Ask,Open,High Low,Close,Volume,"Value, PHP","Buying/(Selling),",Symbol
3,ASIA UNITED AUB,43.85,44.65,44.25,44.8 43.85,44.65,"1,500","66,325",-,AUB
4,BDO UNIBANK BDO,128,129.1,128,129.3 127,128,"3,082,280","395,689,946","5,334,923",BDO
6,BANK PH ISLANDS BPI,94.4,95,97,97 94.4,94.4,"1,659,260","158,274,462.5","(44,899,805)",BPI
7,CHINABANK CHIB,28,28.1,27.75,28 27.65,28,"348,500","9,714,190","1,042,415",CHIB
8,CITYSTATE BANK CSB,6.84,7.99,-,- -,-,-,-,-,CSB


In [178]:
new_df = no_missing.append(ask_missing).append(volume_missing).append(close_missing).append(bid_missing).append(buysell_missing).drop(columns ='Null')

In [179]:
new_df['High'] = np.where(new_df['High'].isna(),new_df['High Low'].apply(lambda x: str(x).split(' ')[0]),new_df['High'])
new_df['Low'] = np.where(new_df['Low'].isna(),new_df['High Low'].apply(lambda x: str(x).split(' ')[-1]),new_df['Low'])
new_df = new_df.rename(columns = {'Buying/(Selling),':'Net Foreign Buying/(Selling), PHP'})

new_df.columns

Index(['Issue Name Symbol', 'Bid', 'Ask', 'Open', 'High Low', 'Close',
       'Volume', 'Value, PHP', 'Net Foreign Buying/(Selling), PHP', 'Symbol',
       'High', 'Low'],
      dtype='object')

In [180]:
new_df = new_df[['Issue Name Symbol','Symbol','Bid', 'Ask', 'Open', 'High', 'Low', 'Close',
       'Volume', 'Value, PHP', 'Net Foreign Buying/(Selling), PHP']].drop_duplicates(subset ='Symbol',keep='first')
new_df

,Issue Name Symbol,Symbol,Bid,Ask,Open,High,Low,Close,Volume,"Value, PHP","Net Foreign Buying/(Selling), PHP"
3,ASIA UNITED AUB,AUB,43.85,44.65,44.25,44.8,43.85,44.65,"1,500","66,325",-
4,BDO UNIBANK BDO,BDO,128,129.1,128,129.3,127,128,"3,082,280","395,689,946","5,334,923"
6,BANK PH ISLANDS BPI,BPI,94.4,95,97,97,94.4,94.4,"1,659,260","158,274,462.5","(44,899,805)"
7,CHINABANK CHIB,CHIB,28,28.1,27.75,28,27.65,28,"348,500","9,714,190","1,042,415"
8,CITYSTATE BANK CSB,CSB,6.84,7.99,-,-,-,-,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...
389,KEPWEALTH KPPI,KPPI,2.54,2.93,- -,-,-,-,-,-,-
390,MAKATI FINANCE MFIN,MFIN,2.16,2.69,- -,-,-,-,-,-,-
391,MERRYMART MM,MM,1.61,1.63,1.63 1.63,1.59,1.59,1.61,"1,242,000","1,999,930","48,210"
392,XURPAS X,X,0.305,0.315,0.31 0.31,0.305,0.305,0.305,"410,000","127,000",-


In [184]:
# Check for missed symbols

missed_symbols = list(set(pse_symbols)- set(new_df['Symbol']))
missed_symbols

[]

## Save Dataframe as a CSV File

Save and Download the new CSV file

In [160]:
pdf_path

'May 06, 2022-EOD.pdf'

In [199]:
# Add Date Column
new_df['Date'] = pdf_path.split('-EOD')[0]

new_df.head()

,Issue Name Symbol,Symbol,Bid,Ask,Open,High,Low,Close,Volume,"Value, PHP","Net Foreign Buying/(Selling), PHP",Date
3,ASIA UNITED AUB,AUB,43.85,44.65,44.25,44.8,43.85,44.65,"1,500","66,325",-,"May 06, 2022"
4,BDO UNIBANK BDO,BDO,128,129.1,128,129.3,127,128,"3,082,280","395,689,946","5,334,923","May 06, 2022"
6,BANK PH ISLANDS BPI,BPI,94.4,95,97,97,94.4,94.4,"1,659,260","158,274,462.5","(44,899,805)","May 06, 2022"
7,CHINABANK CHIB,CHIB,28,28.1,27.75,28,27.65,28,"348,500","9,714,190","1,042,415","May 06, 2022"
8,CITYSTATE BANK CSB,CSB,6.84,7.99,-,-,-,-,-,-,-,"May 06, 2022"


In [ ]:
#Save Dataframe to a csv file
file_name = pdf_path.replace('.pdf','.csv')
new_df.to_csv(file_name,index=False)

In [ ]:
# Download new csv File
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>